# Notebook example

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

from src.data_source.spreadsheet import Spreadsheet
from src.preprocessing import Preprocessing
from src.model import TrainerSklearn

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = Spreadsheet().get_data('../data/raw/train.csv')

In [ ]:
df.columns

In [ ]:
p = Preprocessing()

In [ ]:
df = p.clean_data(df)
df = p.categ_encoding(df)

In [ ]:
df.head()

In [ ]:
X = df.drop(columns=["Survived"])
y = df["Survived"]

In [ ]:
# Ensure the same random state passed to TrainerSkleran().train()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
rf = TrainerSklearn().train(X, y, classification=True, 
                            algorithm=RandomForestClassifier, 
                            preprocessing=p,
                           data_split=('train_test', {'test_size':.3}),
                           random_state=123)

In [ ]:
rf.get_metrics()

In [ ]:
rf.get_columns()

In [ ]:
rf.predict_proba(X_test, binary=True)

In [ ]:
# Predicting new data
def predict_new(X, model, probs=True):
    X = p.clean_data(X)
    X = p.categ_encoding(X)
    
    columns = model.get_columns()
    for col in columns:
        if col not in X.columns:
            X[col] = 0
    print(X)
    if probs:
        return model.predict_proba(X)
    else:
        return model.predict(X)

In [ ]:
new_data = pd.DataFrame({
    'Pclass':3,
    'Sex': 'male',
    'Age':4
}, index=[0])

new_data

In [ ]:
predict_new(new_data, rf)

**Get local explainer for each instance:**

In [ ]:
# Get local explainer
res = rf.local_interpret(X_test, len(X_test.columns))

In [ ]:
res

**Data Quality:**

In [ ]:
from src.preprocessing import DataDrift
import great_expectations as ge

In [ ]:
df = Spreadsheet().get_data('../data/raw/train.csv')

In [ ]:
X_train, X_test = train_test_split(df, test_size=0.3, random_state=123)
X_train.shape, X_test.shape

In [ ]:
dq = DataDrift(discrete_cat_cols=['Sex', 'Pclass'])
df_ge = dq.check(X_train, target='Survived')

In [ ]:
df_ge.save_expectation_suite('../data/output/expectations.json')

In [ ]:
X_test.drop(columns=['Survived'], inplace=True)
df_ge = ge.dataset.PandasDataset(X_test)
ge_val = df_ge.validate(expectation_suite='../data/output/expectations.json', only_return_failures=False)

In [ ]:
ge_val

**Hypothesis testing**

In [ ]:
import pandas as pd
from src.analysis import HTestAutoPilot, HypothesisTester

In [ ]:
df = Spreadsheet().get_data('../data/raw/train.csv')

In [ ]:
survived_age = df[(df.Survived == 1) & (df.Age.notnull())].Age.values
not_survived_age = df[(df.Survived == 0)  & (df.Age.notnull())].Age.values

In [ ]:
# If you want to compare distributions, but you are not sure if the data satisfy the parametric test conditions, 
# you can call the HTestAutoPilot Class.
# It will help you in this decision.
HTestAutoPilot.independent_difference(survived_age, not_survived_age, label1='Survived', label2='Not Survived')

In [ ]:
# If you know which test to use, you can call it directly from the HypothesisTester Class
HypothesisTester.t_test(survived_age, not_survived_age, show_graph=False)

**Dimensionality Reduction**

In [ ]:
import pandas as pd
import numpy as np
from src.analysis import DimensionalityReducer

df = pd.DataFrame({'Col_1': [-1, -2, -3, 1, 2, 3], 'Col_2': [-1, -1, -2, 1, 1, 2], 'Col_3': [-5, -4, -3, 2, 1, 1]})

import pandas as pd
import numpy as np
from src.analysis import DimensionalityReducer

In [ ]:
import pandas as pd
import numpy as np
from src.analysis import DimensionalityReducer

In [ ]:
df = pd.DataFrame({'Col_1': [-1, -2, -3, 1, 2, 3], 'Col_2': [-1, -1, -2, 1, 1, 2], 'Col_3': [-5, -4, -3, 2, 1, 1]})

In [ ]:
df

In [ ]:
dm = DimensionalityReducer(reducer='pca', columns=['Col_1', 'Col_2'], k=1)

In [ ]:
dm.fit_transform(df, y='Col_3')